In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

if groq_api_key:
    os.environ["GROQ_API_KEY"] = groq_api_key

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B5A1680C20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B5A14B4D40>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello, My name is Lore and I am an AI Engineer")])

AIMessage(content="Hello Lore, it's nice to meet you!\n\nThat's fascinating, an AI Engineer! What kind of projects are you working on these days?  Are you building new models, improving existing ones, or perhaps exploring some cutting-edge applications of AI? I'm always eager to learn more about what people are doing in the field.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 21, 'total_tokens': 95, 'completion_time': 0.134545455, 'prompt_time': 0.000142149, 'queue_time': 0.019660739, 'total_time': 0.134687604}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-220b5838-29d9-4666-88e4-dc4811298fd7-0', usage_metadata={'input_tokens': 21, 'output_tokens': 74, 'total_tokens': 95})

In [6]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hello, My name is Lore and I am an AI Engineer"),
    AIMessage(content="Hello Lore, it's nice to meet you!\n\nThat's fascinating, an AI Engineer! What kind of projects are you working on these days?  Are you building new models, improving existing ones, or perhaps exploring some cutting-edge applications of AI? I'm always eager to learn more about what people are doing in the field.\n"),
    HumanMessage(content="Hey, What's my name and what do I do?"),
])

AIMessage(content="You said your name is Lore and that you are an AI Engineer.  😄  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm curious to hear more! \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 116, 'total_tokens': 164, 'completion_time': 0.087272727, 'prompt_time': 0.003606421, 'queue_time': 0.021145035000000003, 'total_time': 0.090879148}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9a317fed-016c-4c1f-b755-208186698ba6-0', usage_metadata={'input_tokens': 116, 'output_tokens': 48, 'total_tokens': 164})

### Messages History
We can use a Message History class to wrap our model and make it stateful. This will kep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [11]:
config={"configurable":{"session_id":"chat1"}}

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hello, My name is Lore and I am an AI Engineer")],
    config=config
)

In [15]:
response.content

"Hello Lore! It's a pleasure to meet you. \n\nBeing an AI Engineer is an exciting field! What kind of projects are you working on these days?  Are you focused on a particular area like natural language processing, computer vision, or something else entirely?  \n\nI'm always interested to hear about the innovative work people are doing in AI.\n"

In [16]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content="Your name is Lore.  \n\nYou told me at the beginning of our conversation! 😊  It's nice to know your name. \n\n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 188, 'total_tokens': 221, 'completion_time': 0.06, 'prompt_time': 0.005512123, 'queue_time': 0.022347704, 'total_time': 0.065512123}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-cfb13683-1f0c-4426-9baa-e853e6b59cb7-0', usage_metadata={'input_tokens': 188, 'output_tokens': 33, 'total_tokens': 221})

In [18]:
## Change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)

response.content

"I still don't know your name!  \n\nYou'll have to tell me.  What's your name? 😊  \n\n"

# Prompt templates
Prompt templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated.
First, let's add in a system message with some custome instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [20]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages([
    ("system","You are helpful assistant. Answer all the questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model


In [21]:
chain.invoke({"messages":[HumanMessage(content="Hello, My name is Lore and I am an AI Engineer")]})

AIMessage(content="Hello Lore, it's nice to meet you! I'm glad you're here.\n\nHow can I be helpful to you today?  Do you have any questions about AI engineering, or perhaps a task you'd like assistance with?  \n\nI'm ready for anything! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 37, 'total_tokens': 103, 'completion_time': 0.12, 'prompt_time': 0.001334707, 'queue_time': 0.020546349999999998, 'total_time': 0.121334707}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b0f3ffc0-c7f4-4d00-9531-8916fbd19a9f-0', usage_metadata={'input_tokens': 37, 'output_tokens': 66, 'total_tokens': 103})

In [23]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [24]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hello, My name is Lore and I am an AI Engineer")],
    config=config
)

response

AIMessage(content="Hello Lore! It's nice to meet you. \n\nI'm happy to be your helpful assistant. What can I do for you today? Do you have any questions about AI engineering, or perhaps something else entirely?  \n\nI'm ready for anything!  😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 37, 'total_tokens': 99, 'completion_time': 0.112727273, 'prompt_time': 0.0003048, 'queue_time': 0.02034546, 'total_time': 0.113032073}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7012a2b0-3310-4ec6-9ed1-6aeeee1d3dc5-0', usage_metadata={'input_tokens': 37, 'output_tokens': 62, 'total_tokens': 99})

In [25]:
response=with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

response.content

'Your name is Lore.  😄  \n\nI remembered from our introduction!  \n\n\n\nWhat can I help you with today, Lore?\n'

In [27]:
## Add more complexity to the prompt
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages([
    ("system","You are helpful assistant. Answer all the questions to the best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [29]:
response=chain.invoke({"messages":[HumanMessage(content="Hello, My name is Lore and I am an AI Engineer")],"language":"Spanish"})
response.content


'¡Hola Lore! Encantado de conocerte.  Soy un asistente útil y haré todo lo posible para responder a tus preguntas en español. ¿Qué puedo hacer por ti? \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [30]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [36]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hello, My name is Lore and I am an AI Engineer")], 'language': 'French'}, 
    config=config
)
response.content

'Olá Lore! É um prazer te conhecer. Como engenheira de IA, com que tipo de projetos você trabalha? \n\n\n\n\n'

In [37]:
response=with_message_history.invoke(
    {'messages': [HumanMessage(content="What's my name?")], 'language': 'French'}, 
    config=config
)
response.content

'Votre nom est Lore.  \n'

# Managing the Conversation History
One important concept to understand when building chatbots is to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

`trim_messages` helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [43]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you'are a good assistant"),
    HumanMessage(content="Hi! I'm Luigi"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like milk shake vanilla"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you'are a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like milk shake vanilla', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [44]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What milkshake do I like?")],
    "language":"English"
    }
)

response.content

"As a large language model, I don't have access to your personal information, including your milkshake preferences.  \n\nWhat's your favorite flavor?  Maybe I can help you brainstorm some milkshake combinations! 😊\n"

In [46]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What math problem did I ask?")],
    "language":"English"
    }
)

response.content

'You asked "What is 2 + 2?".  \n\n\n\nLet me know if you\'d like to try another one! 😊  \n'

In [50]:
## Lets wrap this in the Message History
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [51]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What math problem did I ask?")],
    "language":"English"
    },
    config=config
)

response.content

'You asked: "whats 2 + 2"  \n\nI\'m always ready for more math (or any other kind of) questions! 😄  \n'

In [49]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What milkshake do I like?")],
    "language":"English"
    },
    config=config
)

response.content

"As an AI, I don't have access to past conversations or any information about you.  So I wouldn't know what kind of milkshake you like! \n\nWhat's your favorite flavor? 😊  \n\n\n\n\n"